

# Homework 2

### Course: Information Security

### **Names**
Ericson López

Pablo Herrera

## 1. Standard Digital Signature Protocol Review

The standard, cryptographically robust protocol for creating a digital signature uses asymmetric cryptography combined with secure hashing to provide assurance of both origin authentication and integrity.
The process follows four steps:
- Digest Computation: A cryptographic hash function computes a short digest $H(M)$ of the original message $M$.
- Signing: The sender encrypts the hash value $H(M)$ using their own Private Key ($SK_{Signer}$). This encrypted hash is the signature ($Sig$).
- Transmission: The sender transmits the original message and the signature: $\{M, Sig\}$.
- Verification: The receiver decrypts $Sig$ using the sender's publicly available Public Key ($PK_{Signer}$) to recover the original hash value $H(M)$. The receiver then independently computes a hash $H(M')$ on the received message $M'$ and verifies that $H(M') = H(M)$. If they match, integrity and authentication are confirmed.
 

### Critique of Flawed Alternatives

Students must analyze two proposed alternative signature schemes that deviate from the standard protocol and detail which desirable properties are violated.


### Mechanism 1: 

This scheme proposes using a symmetric key scheme instead of Public Key Cryptography (PKC). To verify the signature, the recipient must give the signature back to the signer, who decrypts and checks it.

#### Answer
**Principles fulfilled by the original system:** Authentication, integrity, and non-repudiation. 

Suppose Bob and Alice agree on a symmetric key (K). Bob (the signer) calculates the message digest (H(M)). He then encrypts this digest with K doing: $S = \text{E}_K(H(M))$. Bob sends ((M, S)) to Alice. 

In this symmetric cryptography scheme, **authentication cannot be guaranteed**: since Alice also knows the key (K), she (or a third party who obtains (K)) could encrypt any summary (H(M')), generate (S'), and claim that Bob signed it. Bob has no way of proving that *he exclusively* generated (S). Therefore, the authentication of the signer is broken. 

At the same time, **non-repudiation is violated**: Bob could argue “I did not produce that (S)”, and Alice has no means of proving that Bob did, because the key (K) is shared and there is no independent evidence of authorship attributable exclusively to Bob. 

Finally, **integrity may also be compromised** in practice: although Bob technically encrypted a hash, Alice does not independently verify that the message received (M) is the same as the one Bob included in the encryption (or that it was not altered before or after). Furthermore, the fact that verification depends on Bob (or the use of the shared key) weakens the guarantee that any manipulation will be detected without relying on Bob.

In summary: this symmetric key mechanism fails in the three principles that the standard system aims to offer (authentication, integrity, non-repudiation) in its strong and independent form.

### Mechanism 2:

The signer encrypts the entire object $M$ with their private key, and the reader decrypts the entire resulting ciphertext using the signer's public key to both read and verify the content.

#### Answer
Applying encryption to the entire message (M) would be very time-consuming, since the cost depends directly on the length of (M). This eliminates the advantage of using hashing, because the hash allows you to have a short “summary” that represents all the information and also serves as a direct mechanism for verifying integrity.

When encrypting the entire message with the private key, that explicit verification step is lost. Therefore, **the principle of integrity is violated**: the recipient can only decrypt using the public key, but has no way of checking whether the (M) they obtain is exactly the original message that the signer intended to protect. There is no independent value (such as the hash) against which to compare. In other words, the recipient obtains a message, but cannot verify whether it was modified before decryption.

Furthermore, if the ciphertext undergoes any alteration during transmission, this could break the decryption, or even produce inconsistent results. This means that the system does not have a formal mechanism for detecting alterations in a clear manner, separate from the process of “attempting to decrypt,” which is not the same as verifying integrity.

Now, regarding **authentication and non-repudiation**: strictly speaking, these could still be maintained because only someone with the private key could generate that ciphertext, and anyone with the public key can verify that it comes from that private key. However, this occurs **because it is asymmetric encryption**, not because it is a valid signing mechanism. In practice, it is a kind of “proof of authorship” due to the fact that it has been encrypted with the private key.

But here's an important nuance: although authentication and non-repudiation could be partially fulfilled by the mathematical property of asymmetric encryption, **this is not a digital signature**. A digital signature involves an explicit signing process (usually on a hash) that allows verification without decrypting the message and produces an independent artifact called a “signature.” In contrast, this mechanism mixes the message with the “proof of authorship” in a single operation, which is not ideal and loses clarity and verifiability.

Therefore, although it can be argued that there is authentication and non-repudiation by accident (a result of how asymmetric cryptography works), in reality **the protocol and principles are not being adequately fulfilled**, and the major principle that is definitely being broken is integrity.

## 2. Vulnerability to Attack Vectors

In the following scenarion, Oscar is an adversary. Provide a detail analysis of the potential problems with each of them, the vulnerabilities that Oscar is exploiting, and the cryptographic solutions that can be used to address them. 


### Scenario 1:

Alice sends to Bob:
-  x = "Transfer \$1000 to Mark" 
- the corresponding digital signature $auth(x)$

Oscar intercepts the transmission and replaces the recipient, modifying the message to:
- x =  "Transfer \$1000 to Oscar". 

Will Bob detect this alteration?

#### Answer
If the previous standard system is used, it should detect it, since when decrypting the signature and hashing (M'), Bob will immediately check that $(H(M) ≠ H(M'))$. This happens because the signature is mathematically linked to the exact content of the message that Alice signed: she actually signed (H(“Transfer $1000 to Mark”)), not the hash of the modified message. So when Bob recalculates the hash of the altered message (“Transfer $1000 to Oscar”) and compares that value with the hash retrieved from the signature, he will see that they do not match, which breaks the verification and blocks Oscar's attempt.

As for the attack analysis, here Oscar is attempting an **integrity attack**, modifying the message content while retaining the original signature, in the hope that Bob will not be able to detect the alteration. If the system did not sign the entire message hash or if Bob did not recomputed that hash, this attack could work. But the cryptographic solution in the standard protocol (signing (H(M)) and forcing the receiver to recalculate (H(M')) and compare it with the signed value) neutralizes precisely this vector, because any change in the text produces a completely different hash, causing (H(M')) to not match the hash that Alice signed. Therefore, even a minimal modification of the message invalidates the signature, and Bob detects the tampering without ambiguity.


### Scenario 2:

Oscar observes a valid transaction message $x$ ("Transfer \$1000 to Mark") and its authentic signature $auth(x)$. Oscar intercepts and retransmits this exact message and signature 100 times to Bob.

#### Answer
In this case, since Oscar does not modify anything and simply resends exactly the same pair ((x, auth(x))), the signature remains completely valid. Bob, when verifying each message, will see that (auth(x)) corresponds to (H(x)) and that the content has not been altered, so from the point of view of the signature system everything “looks correct.” Here, integrity is not compromised because the message is identical, authentication does not fail because the signature does come from Alice's private key, and non-repudiation is maintained because Alice did indeed sign that message. The problem is not with the signature itself, but rather that the system has no way of distinguishing between “the first time Alice wanted to send this instruction” and the 100 malicious forwards made by Oscar. In other words, the system recognizes the message as valid, but does not know that it has seen it before.

The vulnerability Oscar is exploiting here is a **replay attack**: he is reusing an authentic message and its signature to force something that Alice only authorized once to be processed more than once. The security patch for this type of attack is not to change the signature, but to **add a freshness or uniqueness mechanism** within the signed message. This can be a nonce, a unique transaction ID, timestamps, or a sequence counter that Bob verifies. In practice, Alice should sign something like “Transfer $1000 to Mark, tx_id = 8472,” and Bob would keep a record of IDs already processed. With that, even if Oscar forwards the same message 100 times, Bob would automatically reject it as a duplicate. In this way, the protocol maintains authentication and integrity, but prevents a valid signature from being exploited more than once.

##### References
- https://www.ibm.com/docs/en/was/8.5.5?topic=wsspmica-security-considerations-web-services-1
- https://www.ccn-cert.cni.es/publico/seriesCCN-STIC/series/400-Guias_Generales/401-glosario_abreviaturas/topics/97.html
- https://www.1kosmos.com/security-glossary/nonce/

## 3. Research: Transport Layer Security (TLS 1.3)

## The architecture of TLS, and its evolution over the last few years
TLS 1.3 is the latest protocol version of TLS released on August 2018. TLS acronym refers to Transport Layer Security, a protocol used to give trust and safeguarded online communication across all internet. The creation and evolution of this protocol was related to the growth of internet and the new vulnerabilities and network attacks discovered.

At the beggining another protocol was used instead of TLS: SSL (Secure Sockets Layer). The first SSL version (SSL 1.0) was developed in 1994 by Netscape Communications, this was an unreleased version which turned into public at 1995 with SSL 2.0, the first public protocol that can encrypt HTTP traffic. However in just one year SSL 3.0 was published, addressing many security vulnerabilities of SSL 2.0 and also preventing the ability to tamper, forge messages, or eavesdrop on web data communication. 

Some years after new vulnerabilities against SSL 3.0, like POODLE attack (Padding Oracle On Downgraded Legacy Encryption), were discovered. For this reason and the increase of use of SSL 3.0, the IETF (Internet Engineering Task Force) standarized a more secure version of SSL 3.0 called TLS 1.0 at RFC 2246 in January 1999. Also HTTPS (Hypertext Transfer Protocol Secure) started to be the acronym for HTTP communication that used SSL or TLS encryption protocol. 

In 2006, TLS 1.1 was published by IETF in RFC 4346, adding protection against cipher block chaining (CBC) attacks, introducing explicit Initialization Vector (IV) for CBC mode encryption and mitigating certain attacks like BEAST attack (Browser Exploit Against SSL/TLS).

In 2008, TLS 1.2 was published by IETF in RFC 5246. This protocol supports stronger cryptographic algortihms like SHA-256 and AES and also improved handshake mechanism to avoid some attacks.

Finally, in 2018 TLS 1.3 was published by IETF in RFC 8446. TLS 1.3 has improves in security and performance, removing deprecated encryption algorithms, giving more security in handshake mechanisms and simplifying the protocol removing support for older versions.

Along the history developer companies for computers and servers, software developers and Operating Systems developers tend to develop network architecture for the security protocols used in that time, sometimes giving the option to use a legacy security protocol. For this reason, protocols like SSL, TLS 1.0 and TLS 1.1 were declared deprecated and a lot of webpages and enterprises changed to only allow connections that use TLS 1.2 or TLS 1.3.

Actually most of client web browsers and OS use TLS 1.2 but TLS 1.3 continues to be more a more used across all the internet. TLS is one of the most important protocols on internet; it is not only used in HTTP protocol but also can be used to provide security in many communication protocols like Websockets (ws, wss), FTP (File Transfer Protocol), voice data (VoIP), VPNs (SSL/TLS tunnel).
## Cryptographic primitives that are being used

The first part of TLS protocol and the most important in de TLS/SSL handshake that allows client and server to send the simmetric key which will be used along all the duration of the connection, allowing more fast request in REST services since the connection was established or mantaining a secure connection in persistent communications like websockets during all the session. TLS 1.3 architecture has improvements in security and performance removing permanently support of RSA algorithms and allowing it just in certificates signs. In session encryptions TLS only allows perfect secrecy algorithms based Diffle-Hellman exchange (DHE and ECDHE). The image shows the actual process of TLS 1.3 handshake:

![](https://www.thesslstore.com/blog/wp-content/uploads/2018/03/TLS_1_3_Handshake.jpg)

* First of all a SYN (Synchronize) message is sended from the client to tthe server to start the connection. 
* Server answers SYN with an ACK
* Client sends Client Hello that contains information about itself: supported cipher groups (DHE/ECDHE), client public ephimere keys (public keys for each supported group DHE/ECDHE), 32 random bytes called "client random", supported cipher suited (simmetric encryption + hashing algorythm supported), signature algorythms supported (assimetric algorythm + hashing), SNI (Server Name Identity), ALPN extension (Application Layer Protocol Negotiation) for HTTP 1.1 HTTP 2.0.
* Server send Server Hello that countains information about itself: selected cipher suite, key share extension (group selected and server public ephimere key selected), certificate and certificateVerify (transcript of all actual communication of handshake with hash added encrypted using one of the client supported signature algorythms). Finally server send a finished with hash indicating that server is now in changed communication to symmetric.
* If Server doesn't support the selected groups from client, server send HRR (Hello Retry Request) with the selected group by the server.After that the client had to send a Hello Client again with the supported group of the server.
* After Server Hello, client verify the certificate, generates the simmetric keys and send a finished with hash to the server.
* During this handshake all the information isn't ciphered (vulnerabilitie) but the secret key between client and server has perfect forward secrecy (non reuse of private keys, short-lived keys, non-reused keys). The secret key $Z$ is obtained using client private ephimere key with server public ephimere key and is equal to the secret key $Z$ obtained using client public ephimere key with server private ephimere key.

$ Z=\text{DHE/ECDHE}(public_{client},private_{server})=\text{DHE/ECDHE}(public_{server},private_{client}) $


## How the core goals of confidentiality, integrity, and authentication are being satisfied
* Confidentiality: Confidentiality is satisfied with the secret shared key obtained by using a DIffle-Hellman algorithm, where the generated simmetric key and public keys for Diflle-Hellman are only used for that session. Nobody can obtain the shared secret key because the private ephimere keys wasn't shared and the secret shared key can only be obtained using a public key and a ephimere key.
* Integrity: Integrity is satisfied using hashing during all the handshake giving to both client and server a validation mechanism to know that information wasn't modified or corrupted.
* Authentication: TLS 1.3 satifies authentication with server certificates that contains signs valitaded by a trusted authority.
Also another level of authentication is gived during handshake with a full encryption of the handshake transcription using the server private key.

## Explain the modern applications in which TLS is being used
Actually TLS 1.3 is still in mgiration process from TLS 1.2 but some companies already have implemented this protocol. Cibersecurity and IaaS (Infraestructure as a Service) companies like Cloudfare, AWS, Microsoft and Azure already have implementations of this protocol just like big companies like Facebook and Google. The adoption of TLS 1.3 will be a slow process as client and server needs to support DHE/ECDHE encryption algorithms and the new handshake process on the OS or web browser used for the client and the server architecture of a server.

### References
https://www.cloudflare.com/learning/ssl/why-use-tls-1.3

https://www.appviewx.com/blogs/from-ssl-to-tls-1-3-30-years-of-encryption-and-innovation

https://www.geeksforgeeks.org/computer-networks/transport-layer-security-tls-handshake

https://www.geeksforgeeks.org/computer-networks/transport-layer-security-tls

https://www.cloudflare.com/learning/ssl/what-happens-in-a-tls-handshake



## 4. Design Problem

Design an architecture for a system that requires strong, verifiable, and legally enforceable non-repudiation, utilizing cryptographic principles alongside robust protocol design.

### Scenario: 

The application is a Secure Electronic Contract Signing System (SECS). Two parties, Alice (Service Provider) and Bob (Client), must execute a critical digital contract (e.g., an intellectual property license). The system must guarantee Non-Repudiation of Origin (neither party can deny signing the contract) and Non-Repudiation of Submission/Receipt (neither party can deny receiving the final signed contract). This is vital for maintaining the trustworthiness and legal enforceability of the digital transaction.

The proposed architecture
As we need Non-Repudiation is important to establish integrity across the transaction and also Authentication to avoid possible fake identity during the sign.
First of all Bob and Alice will need to have their own identity certificates validated by a trust authority (encryption + hashes). With this certificates, Alice will send Bob the proof that Alice is the Service Provider and then Bob will send Alice the proof that he is the actual Client.

To guarantee the emission/reception during transaction is important to save logs in a third place. In this place all the emissions will be saved as logs (receipt timestamp, document state, emissor entity, receipt entity) with its own hash to guarantee integrity and no modification of that registry.

In each message sent to Alice or to Bob, after the message reach the needed person a third message will be automatically sent to this third place including the brief of that operation of the contract signing.

The third place can be accessed by anyone and can verify the transaction hashing the plain text and validating with the hash stored. Also, to guarantee the persistence of logs audit table of database will be enabled to see if anyone with no permissions tried to modify the registry and validate timestamps en the registry with the real creaation times on the database.

The transaction process to sign the contract using assimetric encryption (RSA) is the following:

* At first Alice sends Bob the contract to sign. Alice sends the contract plus a transcription of actual state of sign encrypted using Alice's certificate private key and the transcription hashed using Alice's certificate hash function.

* Bob recieves the message, validates again the authenticity of the sender and signs the document. After this, Bob resends the contract with sign and a transcription of  actual state of sign encrypted using Bob's certificate private key and the transcription hashed using Bob's certificate hash function.

* Alice recieves the message, validates again the authenticity of the sender and sign her part of the document. After this Alice send the final document to Bob.

* Bob recieves the final document, validates again the authenticity of the sender and sends a confirmation message to Alice (always with hash + encryption).

* Alice recieves the confirmation, validates again the autenticity of the sender and stores the signed document with the document hash in a secure common database (can be stored also in the third place).

Same process can be established with Diffle-Hellman assymetric encryption algorithms. The difference is that Alice and Bob will send the messages using simmetric shared key, knowing that nobody more can known that key unless they have a private ephimere key of Alice or Bob (private ephimere key is not shared so the shared secret key in this algorythm is also guarantees authenticity on both sides).